# ASTEC

Welcome to our ASTEC algorithm

ASTEC (Adaptive Segmentation and Tracking of Embryonic Cells) is a single-pass algorithm, which simultaneously performs cell segmentation and tracking of a 4D sequence of fused data by propagating segmentations between consecutive time points. The pipeline includes

&nbsp; &nbsp; 1. An pre-segmentation phase, during which raw datasets are combined to produce a time stack of 3D fused images

&nbsp; &nbsp; 2. An initialization phase, during which a reference manually-curated segmentation of the first time point is generated

&nbsp; &nbsp; 3. An iteration phase, during which segmentations and trackings are propagated forward in time between consecutive time points up to the end of the sequence

&nbsp; &nbsp; 4. A post-processing phase, during which both cell lineages and segmentations are corrected.

The output of ASTEC consists of a 4D sequence of segmented data as .inr images, and a .pkl file with the embryonic lineage tree.

Each cell in the lineage tree is uniquely identified at each time point by a numeric label $\chi$.

Such a label results from the combination of the index $\nu$ of the cell in the segmented image and the time point $t$ of the image as $\chi=t\cdot 10^4+\nu$.
(For example, the cell with index $4$ at time point $25$ will be labelled as cell $250004$)

The lineage tree is given as a dictionary, whose keys are the numeric labels of cells at all time points but the last one. To each key is associated as a value the numerical label of the same cell at the next time point (if the cell has not divided) or the labels of its two daughters (in the case a division has taken place).


### To run some Python code with Notebooks

To run the code below:

1. Click on the cell to select it.
2. Press `SHIFT+ENTER` on your keyboard or press the play button (<button class='fa fa-play icon-play btn btn-xs btn-default'></button>) in the toolbar above.

A full tutorial for using the notebook interface is available [here](http://jupyter.cs.brynmawr.edu/hub/dblank/public/Jupyter%20Notebook%20Users%20Manual.ipynb).



### Example 
This notebook is a complete example of the package.

In this example ASTEC performs the fusion and the segmentation of a set of raw images taken, over 20 time points, by two cameras of a light-sheet microscope.
Note that these data are a subset of the ones presented in the paper, downsampled in all three dimensions by a factor 2.

This raw dataset can be found in the folder DATA/RAW-ANGLES, while all the functions used in the code below can be found in the ASTEC folder.

DISCLAIMER: ASTEC depends on a number of user-controlled parameters. In this notebook, they are given the same value used to produce the segmented data presented in the paper.
For further details about each of these parameters we refer to the Supplementary Information.



# Data  Specification

In [ ]:
#In order to test the different algorithm you'll find in DATA path an example with 20 time steps.
#Please refer to the specification for DATA input format to try the algorithm on your own data
#DATA DESCRIPTION 
data_path="/home/astec/Desktop/DATA/" # Data path in inr, tiff, tif, hdf5 or zip  format 
delta = 1 # Interval between two time points (if one does not want to fuse every single time point)
ori = 'right' # if im2 angle - im1 angle < 0 => right
resolution = (.34,.34 ,2.) # Resolution of the raw images
delay = 0 # If the time stamps in the folder are not the actual time stamps in the global movie
mirrors = False  #TO COMMENT
target_resolution = .3 # Isotropic resolution of the final fused image


#Image Path definition
path_angle1=data_path+"RAW-ANGLES/LC/Stack0000" # 1st image from the left camera, good quality image at the beginning
path_angle2=data_path+"RAW-ANGLES/RC/Stack0000" # 1st image from the right camera
path_angle3=data_path+"RAW-ANGLES/LC/Stack0001" # 2nd image from the left camera
path_angle4=data_path+"RAW-ANGLES/RC/Stack0001" # 2nd from the right camera

# Import functions

In [ ]:
import sys,os
sys.path.append('ASTEC') #Add the ASTEC Function
sys.path.append('ASTEC/CommunFunctions')
from ImageHandling import imread, imsave, SpatialImage
from FUSION import read_raw_data,fusion_process 
from MARS import mars_segmentation 
from ASTEC import segmentation_propagation
from lineage import write_tlp_from_lin_tree,read_lineage_tree,write_lineage_tree,timeNamed,timesNamed
from post_correction import apply_cell_fusion,remove_too_little_branches

# Data Fusion from different angles

In [ ]:
fused_path=data_path+'FUSED/'# Path to the output folder for fused data
fused_files=fused_path+'Fusion_$TIME.inr' 

#Search for image format in different angle folders
success1,begin1,end1,ext_im1,path_im1=read_raw_data(path_angle1) 
success2,begin2,end2,ext_im2,path_im2=read_raw_data(path_angle2)
success3,begin3,end3,ext_im3,path_im3=read_raw_data(path_angle3) 
success4,begin4,end4,ext_im4,path_im4=read_raw_data(path_angle4) 

if not success1==success2==success3==success4==1 :
     print 'Error in your files, please double check your path files '
elif not begin1==begin2==begin3==begin4:
     print 'Error in your angles file do not start at the same time point'
elif not end1==end2==end3==end4:
    print 'Error in your angles file do not end at the same time point'
elif not ext_im1==ext_im2==ext_im3==ext_im4 :
    print 'Error in your angles file do not have the same extension'
else:
    begin=begin1;end=end1;ext_im=ext_im1;
    print 'Process Fusion from ' + str(begin)+ ' to ' + str(end)
    angles_files=[path_im1, path_im2, path_im3, path_im4] #Combine Angle Path
    temporary_path=fused_path+"TEMP_TIME$TIME/" #Temporary Path
    #PROCESS THE FUSION
    for time in range(begin, end+1, delta): # Interation on time steps
        fused_file=timeNamed(fused_files,time+delay)
        if not os.path.isfile(fused_file):
            time_angles_files=[timeNamed(angle_file + ext_im,time) for angle_file in angles_files]
            temporary_time_path=timeNamed(temporary_path,time) #Temporary Path for this time point
            time_process=fusion_process(time_angles_files,
                       fused_file,  
                       temporary_time_path,
                       ori, resolution,target_resolution, delay,
                       ext_im1, mirrors = mirrors)
            print "Time point " + str(time) + " takes " + str(time_process) + " to compute\n\n\n"
            os.system("rm -rf "+temporary_time_path) ### Cleaning temporary files
        


# Segmentation of the first time point using MARS algorithm

In [ ]:
fused_file =timeNamed(fused_files,begin)  #First Fused time step
segmentation_path=data_path+'SEGMENTATION/' # Path to the output folder for segmented data
segmentation_files=segmentation_path+'Segmented_t$TIME.inr' #Segmentation output files
#Parameters for MARS segmentation
sigma1 = 0.6 / target_resolution   #sigma 1 (0.6um)
sigma2 = 0.15 / target_resolution #sigma 2 (0.15um) 
h_min = 4   # H min initialisation to ease correction
         
segmentation_file = timeNamed(segmentation_files,begin) #First time step to segment
mars_segmentation(fused_file, segmentation_file, sigma1, h_min, sigma2) #Apply Automatic MARS Segmentation 
imsave(segmentation_file.replace('.inr','_save.inr'), imread(segmentation_file)) # Save into segmentation file as inr


# Manual  correction of the segmentation


In [ ]:
import numpy as np
seg = imread(segmentation_file)
mapping = np.arange(np.max(seg)+1)
mapping[28]=1
mapping[7]=8
mapping[4]=9
mapping[6]=10
mapping[14]=15
mapping[22]=13
mapping[27]=20
mapping[30]=25
mapping[41]=32
mapping[39]=44
mapping[50]=46
mapping[52]=45
mapping[62]=58
mapping[59]=64
mapping[63]=66
mapping[71]=79
mapping[75]=69
mapping[76]=83
mapping[84]=77


seg_corrected = mapping[seg]
imsave(segmentation_file, SpatialImage(seg_corrected).astype(np.uint16)) # Save into segmentation file as inr

# ASTEC  segmentation propagation

In [ ]:
### Parameters:
h_min_min=2 # Minimal tested value of h for seed detection
h_min_max=18 # Maximal tested value of h for seed detection
Volum_Min_No_Seed=100 # Minimal volume (in voxels) for a seed to be used in watershed
Thau= 25 # Threshold for detection of cell division during watershed (the higher thau, the stricter the condition) 
MinVolume=1000 # Volume threshold (in voxels) below which cells are erased
VolumeRatioBigger=0.5#If a cell in Sˇt+1 is at least 50% bigger than its progeny in Sˆt+1, 
VolumeRatioSmaller=0.1 #Cells in Sˆt+1 that are 10% or more smaller than their equivalent in Sˇt+1 are tagged for correction. 
MorphosnakeIterations=10 #Iterations for each step of an active contour (Morphosnake) algorithm which uses the dilated shape of cˆ
RadiusOpening=20 # Radius of the sphere used in the Morphosnake algorithm as a structuring element for the opening operation
NIterations=200 # Maximum number of steps of the algorithm (steps can be fewer if stability is reched) 
DeltaVoxels=10**3  # Condition for stability: volume converges at ±δ voxels
  
nb_proc=20 # Number of processor used in parallel computations


lineage_tree_filename=data_path+'lin_tree_information.pkl' #Name of the output file with the lineage tree 
lin_tree_information=read_lineage_tree(lineage_tree_filename,begin) # Read the lineage tree (in case it was previously created)

temporary_folder=segmentation_path+'TEMP/' #  TEMPORARY FOLDER

### PROCESS PROPAGATION SEGMENTATION 
for t in range(begin, end):
    time_segment=t+delta #Time point of Segmentation 
    print 'Starting the segmentation at ' + str(time_segment)
    fused_file_ref=timeNamed(fused_files,t) #Previous image file
    fused_file=timeNamed(fused_files,time_segment) #Actual image file to segment
    segmentation_file_ref=timeNamed(segmentation_files,t) #Previous Segmentation file
    segmentation_file=timeNamed(segmentation_files,time_segment) #Output Segmentation file

    os.system("mkdir -p " + temporary_folder )#  TEMPORARY FOLDER
    vf_file=timesNamed(temporary_folder+'VF_t$TIME1_on_t$TIME2.inr','$TIME1',t,'$TIME2',time_segment) #VECTOR FIELDS FILE
    h_min_files=timeNamed(temporary_folder+'h_min_t$TIME_h$HMIN_s$SIGMA.inr.gz',time_segment)  #HMIN FILES
    seed_file=timeNamed(temporary_folder+'Seed_t$TIME.inr',t) #SEEDS FILE

    #PROCESS PROGATION SEGMENTATION
    seg_from_opt_h, lin_tree_information=segmentation_propagation(t,fused_file_ref,segmentation_file_ref, fused_file, seed_file,vf_file , h_min_files, h_min_min,h_min_max, sigma1, lin_tree_information, delta, nb_proc,RadiusOpening=RadiusOpening,Thau=Thau,MinVolume=MinVolume,VolumeRatioBigger=VolumeRatioBigger,VolumeRatioSmaller=VolumeRatioSmaller,MorphosnakeIterations=MorphosnakeIterations,NIterations=NIterations,DeltaVoxels=DeltaVoxels)
    #SAVE OUTPUT
    print 'Write the segmentation in ' + segmentation_file
    imsave(segmentation_file, seg_from_opt_h)
    write_lineage_tree(lineage_tree_filename,lin_tree_information) #Save the current lineage tree
    os.system("rm -rf  " + temporary_folder ) #DELETE TEMPORATY FILES


# Lineage Tree Post-correction

In [ ]:
### PARAMETERS
Volume_Threshold=10000 # volume lower threshold for final cells (in voxels) 
Soon=True #True if the cell life span has to be taken into account in post-correction
ShortLifespan=25 # Lower threshold (in time points) for the identification of problematically short-lived cells
PearsonThreshold=0.9; # Parameter for volume anti-correlation (Pearson correlation under −0.9)

### CORRECT THE LINEAGE TREE WITH THE CELLS VOLUMES
lin_tree_cor, new_volumes, to_fuse, been_fused=remove_too_little_branches(lin_tree_information['lin_tree'], lin_tree_information['volumes_information'], Volume_Threshold, soon=Soon)

### APPLYING THE CORRECTION ON THE IMAGES
apply_cell_fusion(lin_tree_information['lin_tree'],lin_tree_information['volumes_information'], to_fuse,segmentation_files,segmentation_files,begin, end, delta,)

### SAVE THE FINAL LINEAGE TREE
write_lineage_tree(lineage_tree_filename,lin_tree_cor)
